In [1]:
import os
import glob
import netCDF4
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature

from matplotlib.collections import LineCollection
from matplotlib.path import Path
import matplotlib.path as mpath
import matplotlib.patches as patches
from matplotlib.image import imread
from matplotlib.colors import ListedColormap, BoundaryNorm

from scipy import stats

In [77]:
#################
# Load data with n files
#################
#Path
os.chdir(r'D:\Bigfiles\CEMIP6_ZIL\Historical')
savefilenames = ['AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2-WACCM', 'CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']
n_models = np.size(savefilenames)

#for i in range(18,n_models):
for i in range(0,1):
    print(i)
    print(savefilenames[i])
    savefile_name = savefilenames[i]
    g = 'gn'
    filenames = 'tas_Amon_' + savefile_name + '*.nc'

    ncdf = netCDF4.Dataset(glob.glob(filenames)[0])
    lons = ncdf.variables['lon'][:]
    lats = ncdf.variables['lat'][:]

    tas = np.empty((0,np.size(lats),np.size(lons)))

    for filename in glob.glob(filenames):
        print(filename)
        #Load netcdf
        ncdf = netCDF4.Dataset(filename)
        #print(ncdf.variables.keys())
        #print(ncdf.variables['tas'])
        #print(ncdf.variables['time'])
        #print(ncdf.variables['time'][:])

        #Get dimensions
        lons = ncdf.variables['lon'][:]
        lats = ncdf.variables['lat'][:]
        times = ncdf.variables['time'][:]

        #Calculate datetime
        dtimes = np.empty(len(times), dtype='datetime64[D]')
        for i,t in enumerate(times):
            if savefile_name in ['CESM2-WACCM', 'FGOALS-f3-L', 'NorESM2-MM', 'TaiESM1']:
                dtimes[i] = np.datetime64('0001-01-01') + np.timedelta64(np.int64(t),'D')
            else:
                dtimes[i] = np.datetime64('1850-01-01') + np.timedelta64(np.int64(t),'D')
        dtimes = dtimes.astype('datetime64')

        #Get variables
        tas_n = ncdf.variables['tas'][:,:,:]-273.15
        tas = np.concatenate((tas, tas_n), axis=0)


    #################
    # Get mean yearly array for season
    #################
    #Get season of each dtime
    month_to_season_lu = np.array([
        None,
        'DJF', 'DJF',
        'MAM', 'MAM', 'MAM',
        'JJA', 'JJA', 'JJA',
        'SON', 'SON', 'SON',
        'DJF'
    ])
    grp_season = month_to_season_lu[pd.to_datetime(dtimes).month]

    #Get year of each dtime (where December is the next year)
    grp_year = np.empty(np.size(grp_season))
    for i,date in enumerate(zip(pd.to_datetime(dtimes).year, pd.to_datetime(dtimes).month)):
        if date[1] == 12:
            grp_year[i] = date[0]+1
        else:
            grp_year[i] = date[0]

    #Average data array by season and year
    years_arr = range(1957,2015)
    months_to_end = np.arange(-781, 1, 12)
#    months_to_end = np.aragne
    tas_ANN = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_DJF = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_MAM = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_JJA = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_SON = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])

    for i,year in enumerate(years_arr):    
        tas_DJF[i,:,:] = np.mean(tas[months_to_end[i]:months_to_end[i]+3,:,:], axis=0)
        tas_MAM[i,:,:] = np.mean(tas[months_to_end[i]+3:months_to_end[i]+6,:,:], axis=0)
        tas_JJA[i,:,:] = np.mean(tas[months_to_end[i]+6:months_to_end[i]+9,:,:], axis=0)
        tas_SON[i,:,:] = np.mean(tas[months_to_end[i]+9:months_to_end[i]+12,:,:], axis=0)
        if i == 64:
            tas_ANN[i,:,:] = np.mean(tas[months_to_end[i]:,:,:], axis=0)
        else:
            tas_ANN[i,:,:] = np.mean(tas[months_to_end[i]+1:months_to_end[i]+13,:,:], axis=0)


    #################
    # Save files
    #################
    #ANN
    f = netCDF4.Dataset('DATA-'+savefile_name+'-ANN-historical-1950-2014.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_ANN

    #Add global attributes
    f.description = "CMIP6 historical Annual temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #DJF
    f = netCDF4.Dataset('DATA-'+savefile_name+'-DJF-historical-1950-2014.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_DJF

    #Add global attributes
    f.description = "CMIP6 historical DJF temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #MAM
    f = netCDF4.Dataset('DATA-'+savefile_name+'-MAM-historical-1950-2014.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_MAM

    #Add global attributes
    f.description = "CMIP6 historical MAM temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celcius'

    f.close()

    #JJA
    f = netCDF4.Dataset('DATA-'+savefile_name+'-JJA-historical-1950-2014.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_JJA

    #Add global attributes
    f.description = "CMIP6 historical JJA temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #SON
    f = netCDF4.Dataset('DATA-'+savefile_name+'-SON-historical-1950-2014.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_SON

    #Add global attributes
    f.description = "CMIP6 historical temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

0
AWI-CM-1-1-MR
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195001-195012.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195101-195112.nc


<ipython-input-77-61dc5bb3ae2a>:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lons = ncdf.variables['lon'][:]
<ipython-input-77-61dc5bb3ae2a>:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lats = ncdf.variables['lat'][:]
<ipython-input-77-61dc5bb3ae2a>:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence

tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195201-195212.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195301-195312.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195401-195412.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195501-195512.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195601-195612.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195701-195712.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195801-195812.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195901-195912.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196001-196012.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196101-196112.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196201-196212.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196301-196312.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196401-196412.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196501-196512.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196601-196612.nc
tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_196701-19

In [12]:
#################
# Load data with n files
#################
#Path
os.chdir(r'D:\Bigfiles\CEMIP6_ZIL\SSP245')
savefilenames = ['AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2-WACCM', 'CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']
n_models = np.size(savefilenames)

for i in range(n_models):
    print(i)
    print(savefilenames[i])
    savefile_name = savefilenames[i]
    g = 'gn'
    filenames = 'tas_Amon_' + savefile_name + '*.nc'

    ncdf = netCDF4.Dataset(glob.glob(filenames)[0])
    lons = ncdf.variables['lon'][:]
    lats = ncdf.variables['lat'][:]

    tas = np.empty((0,np.size(lats),np.size(lons)))

    for filename in glob.glob(filenames):
        print(filename)
        #Load netcdf
        ncdf = netCDF4.Dataset(filename)
        #print(ncdf.variables.keys())
        #print(ncdf.variables['tas'])
        #print(ncdf.variables['time'])
        #print(ncdf.variables['time'][:])

        #Get dimensions
        lons = ncdf.variables['lon'][:]
        lats = ncdf.variables['lat'][:]
        times = ncdf.variables['time'][:]

        #Calculate datetime
        dtimes = np.empty(len(times), dtype='datetime64[D]')
        for i,t in enumerate(times):
            if savefile_name in ['CESM2-WACCM', 'FGOALS-f3-L', 'NorESM2-MM', 'TaiESM1']:
                dtimes[i] = np.datetime64('0001-01-01') + np.timedelta64(np.int64(t),'D')
            else:
                dtimes[i] = np.datetime64('1850-01-01') + np.timedelta64(np.int64(t),'D')
        dtimes = dtimes.astype('datetime64')

        #Get variables
        tas_n = ncdf.variables['tas'][:,:,:]-273.15
        tas = np.concatenate((tas, tas_n), axis=0)


    #################
    # Get mean yearly array for season
    #################
    #Get season of each dtime
    month_to_season_lu = np.array([
        None,
        'DJF', 'DJF',
        'MAM', 'MAM', 'MAM',
        'JJA', 'JJA', 'JJA',
        'SON', 'SON', 'SON',
        'DJF'
    ])
    grp_season = month_to_season_lu[pd.to_datetime(dtimes).month]

    #Get year of each dtime (where December is the next year)
    grp_year = np.empty(np.size(grp_season))
    for i,date in enumerate(zip(pd.to_datetime(dtimes).year, pd.to_datetime(dtimes).month)):
        if date[1] == 12:
            grp_year[i] = date[0]+1
        else:
            grp_year[i] = date[0]

    #Average data array by season and year
    years_arr = range(2015,2099)
    months_to_end = np.arange(-1009, 1, 12)
    tas_ANN = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_DJF = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_MAM = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_JJA = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_SON = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])

    for i,year in enumerate(years_arr):
        tas_DJF[i,:,:] = np.mean(tas[months_to_end[i]:months_to_end[i]+3,:,:], axis=0)
        tas_MAM[i,:,:] = np.mean(tas[months_to_end[i]+3:months_to_end[i]+6,:,:], axis=0)
        tas_JJA[i,:,:] = np.mean(tas[months_to_end[i]+6:months_to_end[i]+9,:,:], axis=0)
        tas_SON[i,:,:] = np.mean(tas[months_to_end[i]+9:months_to_end[i]+12,:,:], axis=0)
        if i == 83:
            tas_ANN[i,:,:] = np.mean(tas[months_to_end[i]:,:,:], axis=0)
        else:
            tas_ANN[i,:,:] = np.mean(tas[months_to_end[i]+1:months_to_end[i]+13,:,:], axis=0)


    #################
    # Save files
    #################
    #ANN
    f = netCDF4.Dataset('DATA-'+savefile_name+'-ANN-ssp245-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_ANN

    #Add global attributes
    f.description = "CMIP6 SSP245 Annual temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #DJF
    f = netCDF4.Dataset('DATA-'+savefile_name+'-DJF-ssp245-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_DJF

    #Add global attributes
    f.description = "CMIP6 SSP245 DJF temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #MAM
    f = netCDF4.Dataset('DATA-'+savefile_name+'-MAM-ssp245-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_MAM

    #Add global attributes
    f.description = "CMIP6 SSP245 MAM temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celcius'

    f.close()

    #JJA
    f = netCDF4.Dataset('DATA-'+savefile_name+'-JJA-ssp245-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_JJA

    #Add global attributes
    f.description = "CMIP6 SSP245 JJA temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #SON
    f = netCDF4.Dataset('DATA-'+savefile_name+'-SON-ssp245-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_SON

    #Add global attributes
    f.description = "CMIP6 SSP245 temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

0
AWI-CM-1-1-MR
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_201501-201512.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_201601-201612.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_201701-201712.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_201801-201812.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_201901-201912.nc

<ipython-input-12-f7bd5fc13dbd>:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lons = ncdf.variables['lon'][:]
<ipython-input-12-f7bd5fc13dbd>:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lats = ncdf.variables['lat'][:]
<ipython-input-12-f7bd5fc13dbd>:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence


tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202001-202012.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202101-202112.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202201-202212.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202301-202312.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202401-202412.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202501-202512.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202601-202612.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202701-202712.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202801-202812.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_202901-202912.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203001-203012.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203101-203112.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203201-203212.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203301-203312.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203401-203412.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203501-203512.nc
tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_203601-203612

<ipython-input-12-f7bd5fc13dbd>:46: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  tas_n = ncdf.variables['tas'][:,:,:]-273.15


tas_Amon_CESM2-WACCM_ssp245_r1i1p1f1_gn_206501-210012.nc
-1009
-997
-985
-973
-961
-949
-937
-925
-913
-901
-889
-877
-865
-853
-841
-829
-817
-805
-793
-781
-769
-757
-745
-733
-721
-709
-697
-685
-673
-661
-649
-637
-625
-613
-601
-589
-577
-565
-553
-541
-529
-517
-505
-493
-481
-469
-457
-445
-433
-421
-409
-397
-385
-373
-361
-349
-337
-325
-313
-301
-289
-277
-265
-253
-241
-229
-217
-205
-193
-181
-169
-157
-145
-133
-121
-109
-97
-85
-73
-61
-49
-37
-25
-13
5
CIESM
tas_Amon_CIESM_ssp245_r1i1p1f1_gr_201501-210012.nc
-1009
-997
-985
-973
-961
-949
-937
-925
-913
-901
-889
-877
-865
-853
-841
-829
-817
-805
-793
-781
-769
-757
-745
-733
-721
-709
-697
-685
-673
-661
-649
-637
-625
-613
-601
-589
-577
-565
-553
-541
-529
-517
-505
-493
-481
-469
-457
-445
-433
-421
-409
-397
-385
-373
-361
-349
-337
-325
-313
-301
-289
-277
-265
-253
-241
-229
-217
-205
-193
-181
-169
-157
-145
-133
-121
-109
-97
-85
-73
-61
-49
-37
-25
-13
6
CMCC-CM2-SR5
tas_Amon_CMCC-CM2-SR5_ssp245_r1i1p1f1_gn_20

tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_203701-203712.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_203801-203812.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_203901-203912.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204001-204012.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204101-204112.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204201-204212.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204301-204312.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204401-204412.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204501-204512.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204601-204612.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204701-204712.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204801-204812.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_204901-204912.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_205001-205012.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_205101-205112.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_205201-205212.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_205301-205312.nc
tas_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_205401-205412.nc
tas_Amon_E

tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_208701-208712.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_208801-208812.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_208901-208912.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209001-209012.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209101-209112.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209201-209212.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209301-209312.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209401-209412.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209501-209512.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209601-209612.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209701-209712.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209801-209812.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_209901-209912.nc
tas_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_210001-210012.nc
-1009
-997
-985
-973
-961
-949
-937
-925
-913
-901
-889
-877
-865
-853
-841
-829
-817
-805
-793
-781
-769
-757
-745
-733
-721
-709
-697
-685
-673
-661
-649
-637
-625
-613
-60

In [16]:
#################
# Load data with n files
#################
#Path
os.chdir(r'D:\Bigfiles\CEMIP6_ZIL\SSP585')
savefilenames = ['AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2-WACCM', 'CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']
n_models = np.size(savefilenames)

for i in range(n_models):
    print(i)
    print(savefilenames[i])
    savefile_name = savefilenames[i]
    g = 'gn'
    filenames = 'tas_Amon_' + savefile_name + '*.nc'

    ncdf = netCDF4.Dataset(glob.glob(filenames)[0])
    lons = ncdf.variables['lon'][:]
    lats = ncdf.variables['lat'][:]

    tas = np.empty((0,np.size(lats),np.size(lons)))

    for filename in glob.glob(filenames):
        print(filename)
        #Load netcdf
        ncdf = netCDF4.Dataset(filename)
        #print(ncdf.variables.keys())
        #print(ncdf.variables['tas'])
        #print(ncdf.variables['time'])
        #print(ncdf.variables['time'][:])

        #Get dimensions
        lons = ncdf.variables['lon'][:]
        lats = ncdf.variables['lat'][:]
        times = ncdf.variables['time'][:]

        #Calculate datetime
        dtimes = np.empty(len(times), dtype='datetime64[D]')
        for i,t in enumerate(times):
            if savefile_name in ['CESM2-WACCM', 'FGOALS-f3-L', 'NorESM2-MM', 'TaiESM1']:
                dtimes[i] = np.datetime64('0001-01-01') + np.timedelta64(np.int64(t),'D')
            else:
                dtimes[i] = np.datetime64('1850-01-01') + np.timedelta64(np.int64(t),'D')
        dtimes = dtimes.astype('datetime64')

        #Get variables
        tas_n = ncdf.variables['tas'][:,:,:]-273.15
        tas = np.concatenate((tas, tas_n), axis=0)


    #################
    # Get mean yearly array for season
    #################
    #Get season of each dtime
    month_to_season_lu = np.array([
        None,
        'DJF', 'DJF',
        'MAM', 'MAM', 'MAM',
        'JJA', 'JJA', 'JJA',
        'SON', 'SON', 'SON',
        'DJF'
    ])
    grp_season = month_to_season_lu[pd.to_datetime(dtimes).month]

    #Get year of each dtime (where December is the next year)
    grp_year = np.empty(np.size(grp_season))
    for i,date in enumerate(zip(pd.to_datetime(dtimes).year, pd.to_datetime(dtimes).month)):
        if date[1] == 12:
            grp_year[i] = date[0]+1
        else:
            grp_year[i] = date[0]

    #Average data array by season and year
    years_arr = range(2015,2099)
    months_to_end = np.arange(-1009, 1, 12)
    tas_ANN = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_DJF = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_MAM = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_JJA = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])
    tas_SON = np.empty([np.size(years_arr),np.size(tas,1),np.size(tas,2)])

    for i,year in enumerate(years_arr):
        tas_DJF[i,:,:] = np.mean(tas[months_to_end[i]:months_to_end[i]+3,:,:], axis=0)
        tas_MAM[i,:,:] = np.mean(tas[months_to_end[i]+3:months_to_end[i]+6,:,:], axis=0)
        tas_JJA[i,:,:] = np.mean(tas[months_to_end[i]+6:months_to_end[i]+9,:,:], axis=0)
        tas_SON[i,:,:] = np.mean(tas[months_to_end[i]+9:months_to_end[i]+12,:,:], axis=0)
        if i == 83:
            tas_ANN[i,:,:] = np.mean(tas[months_to_end[i]:,:,:], axis=0)
        else:
            tas_ANN[i,:,:] = np.mean(tas[months_to_end[i]+1:months_to_end[i]+13,:,:], axis=0)


    #################
    # Save files
    #################
    #ANN
    f = netCDF4.Dataset('DATA-'+savefile_name+'-ANN-ssp585-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_ANN

    #Add global attributes
    f.description = "CMIP6 SSP585 Annual temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #DJF
    f = netCDF4.Dataset('DATA-'+savefile_name+'-DJF-ssp585-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_DJF

    #Add global attributes
    f.description = "CMIP6 SSP585 DJF temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #MAM
    f = netCDF4.Dataset('DATA-'+savefile_name+'-MAM-ssp585-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_MAM

    #Add global attributes
    f.description = "CMIP6 SSP585 MAM temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celcius'

    f.close()

    #JJA
    f = netCDF4.Dataset('DATA-'+savefile_name+'-JJA-ssp585-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_JJA

    #Add global attributes
    f.description = "CMIP6 SSP585 JJA temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

    #SON
    f = netCDF4.Dataset('DATA-'+savefile_name+'-SON-ssp585-2015-2099.nc','w', format='NETCDF4')    
    f.createDimension('lons', len(lons))
    f.createDimension('lats', len(lats))
    f.createDimension('times', None)

    longitude = f.createVariable('lons', 'f4', 'lons')
    latitude = f.createVariable('lats', 'f4', 'lats') 
    time = f.createVariable('years', 'i4', 'times')
    temp = f.createVariable('tas', 'f4', ('times', 'lats', 'lons'))

    longitude[:] = lons #The "[:]" at the end of the variable instance is necessary
    latitude[:] = lats
    time[:] = years_arr
    temp[:,:,:] = tas_SON

    #Add global attributes
    f.description = "CMIP6 SSP585 temperature from " + savefile_name
    longitude.units = 'degrees east'
    latitude.units = 'degrees north'
    time.units = 'years'
    temp.units = 'Celsius'

    f.close()

0
AWI-CM-1-1-MR
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_201501-201512.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_201601-201612.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_201701-201712.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_201801-201812.nc

<ipython-input-16-87dc42565678>:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lons = ncdf.variables['lon'][:]
<ipython-input-16-87dc42565678>:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lats = ncdf.variables['lat'][:]
<ipython-input-16-87dc42565678>:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence


tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_201901-201912.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202001-202012.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202101-202112.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202201-202212.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202301-202312.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202401-202412.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202501-202512.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202601-202612.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202701-202712.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202801-202812.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_202901-202912.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_203001-203012.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_203101-203112.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_203201-203212.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_203301-203312.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_203401-203412.nc
tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_203501-203512

<ipython-input-16-87dc42565678>:46: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  tas_n = ncdf.variables['tas'][:,:,:]-273.15


5
CIESM
tas_Amon_CIESM_ssp585_r1i1p1f1_gr_201501-210012.nc
6
CMCC-CM2-SR5
tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc
7
CMCC-ESM2
tas_Amon_CMCC-ESM2_ssp585_r1i1p1f1_gn_201501-210012.nc
8
EC-Earth3
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_201501-201512.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_201601-201612.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_201701-201712.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_201801-201812.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_201901-201912.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202001-202012.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202101-202112.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202201-202212.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202301-202312.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202401-202412.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202501-202512.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202601-202612.nc
tas_Amon_EC-Earth3-Veg_ssp585_r1i1p1f1_gr_202701-202712.nc
tas_Amon_EC-Earth3-Veg

tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_206801-206812.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_206901-206912.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207001-207012.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207101-207112.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207201-207212.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207301-207312.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207401-207412.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207501-207512.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207601-207612.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207701-207712.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207801-207812.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_207901-207912.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_208001-208012.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_208101-208112.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_208201-208212.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_208301-208312.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_208401-208412.nc
tas_Amon_EC-Earth3_ssp585_r1i1p1f1_gr_208501-208512.nc
tas_Amon_E